The two examples we did before are examples of physical systems with behavior that can be calculated more or less exactly, according to well-defined equations. But what about chaotic or complex systems where we don't necessarily have a precise mathematical description of how the system evolves?

Specifically, let's consider systems of large numbers of autonomous agents who move independently but can influence each other. This kind of situation is extremely common in biology and sociology when examining the behavior of animals and humans in groups.

<img width="500px" src="images/bird_flock.gif">
<img width="500px" src="images/traffic.gif">
<img width="500px" src="images/fish.gif">

So how can we model this? It would be incredibly difficult to invidually simulate the motivations of each agent (plus, it would require algorithmic knowledge beyond the scope of this two-hour course). We want a simple system that can handle a lot of agents, but still captures the essential qualities of the system. It turns out that this isn't as hard as it sounds! We can abstract away a lot of the complexity behind what drives real organisms and still obtain the same emergent properties when we lump a lot of them together.

The simulation we're going to build is a very general model of flocking movement: it could be fish, birds, people who like to move in groups, etc. How do we represent an agent? 
Let's make a few assumptions. We assume the agents are identical and stateless - they all react uniformly and don't remember anything about their environment. This means we don't have to store any information about the agent itself beyond universal parameters for all agents. This helps us a lot!

Now we can simplify the situation down to something very similar to the gravity simulation! Each agent only has to interact with the other agents and environment around it, so we only need to store a velocity and position for each agent.

Let 
$$\vec{r_i} = \vec{r_i}(t)$$
$$\vec{v_i} = \vec{v_i}(t)$$ 
designate the position and velocity respectively of bird $i$ as a function of time. Just as with the gravity section, we can use a force to produce an acceleration that changes the velocity: the velocity in turn changes the position. We can still use Euler integration to approximate the continuous position and acceleration functions. The theoretical treatment is quite simple!

Let's make a representation for a group of birds as a list of such [$\vec{r}$, $\vec{v}$] pairs


In [4]:
#This list represents two birds: one with position (10,20) and velocity (3,4)
#the other has position (50,50) and velocity (-1,-2)
birds = [[[10,20],[3,4]],
         [[50,50],[-1,-2]]]
#This is another two birds
morebirds = [[[40,50],[-2,1]],
             [[75,25],[5,6]]]

#Now we make a list of lists of both sets of birds. This creates two "flocks" of birds with two birds in each.
#This is the canonical representation which we will work with
flocks = [birds, morebirds]

But wait! Where do these forces come from? Obviously we don't have a direct force acting on birds to flock together. We choose the forces as we develop the model to approximate real world behavior. The difficulty is choosing forces that produce an accurate reflection of reality.

Let's start with a really simple model. We have a square box 100x100 units and we put a bunch of birds in it. For now, the birds don't do anything special. They start with a random position and velocity.

To start, let's try to keep the birds inside the box! How do we do this? Well, if a bird gets close to a wall it will see the wall and try to turn away so it doesn't crash into the wall. We can simulate this by adding a force that turns the bird away from the wall if it gets too close. Run the boxes below to setup a function that calculate this force for a set of birds.


**Be sure to run the next three cells below (by pressing Ctrl-Enter in each cell) or else everything below this won't work!**


In [2]:
#Import libraries to help us with animation and calculation
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import random

"""
Generates forces that cause birds to avoid running into the walls
"""
def avoid_wall_force(flocks):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    #These constants determine how quickly the bird turns as it gets close to the wall
    c = 1000.0
    d = 50.0
    #Determine how far away the bird is when it reacts. We need to set a minimum distance so the bird has time to turn!
    minv = max(2, min(sight_radius, 5))

    #Inner function to calculate the force on the bird from the walls
    #Python lets you nest functions!
    #This will calculate the force on a single bird from the walls
    def calculate_wall_force(bird):
        force = [0,0]
        #If the bird is too close to the left edge
        if bird[0][0] < minv:
            force[0] += c/bird[0][0]**2
        #If the bird is too close to the right edge
        if 100 - bird[0][0] < minv:
            force[0] -= c/(100-bird[0][0])**2
        #If the bird is too close to the bottom edge
        if bird[0][1] < minv:
            force[1] += c/bird[0][1]**2
        #If the bird is too close to the top edge
        if 100 - bird[0][1] < minv:
            force[1] -= c/(100-bird[0][1])**2
        return force

    #Update the force matrix with the forces we calculated above
    for i in xrange(len(flocks)):
        forces[i] = map(calculate_wall_force, flocks[i])
    
    return forces

Birds also like to keep moving, so let's also add a force that will try to keep them moving at around a constant fraction of their max speed if no other forces are acting on them.

In [3]:
"""Birds always want to be in motion: this force will drive them to try to maintain a constant flight speed of p*max_vel"""
def motive_force(flocks):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    c = 10.0
    p = 0.75
    for i in xrange(len(flocks)):
        forces[i] = map(lambda bird: np.multiply(c*p*bird_max_vel/np.linalg.norm(bird[1]), bird[1]), flocks[i])

    return forces

Let's also put some boilerplate code up here to help us generate and render the birds correctly. 

In [4]:
#Visibility flag to help us render the birds
are_visible = 0

"""This calculates the sum of an arbitrary number of arrays. Used for totalling the force arrays later"""
def sum_arrays(*arrays):
    if len(arrays) == 1:
        return arrays[0]
    total = arrays[0]
    for array in arrays[1:]:
        total = np.add(total, array)
    return total

"""This function updates the velocity and position of a set of agents based on the forces that are applied
'patches' are just rendering objects used by matplotlib: don't worry about what happens to them"""
def updatePositionVelocity(agents, forces, patches, npatches):
    for i,flock in enumerate(agents):
        #Update position and velocities using velocity and force: assume birds have mass 1
        positions = np.add(map(lambda bird: bird[0], flock), np.multiply(interval/1000.0,map(lambda bird: bird[1], flock)))
        velocities = np.add(map(lambda bird: bird[1], flock), np.multiply(interval/1000.0,forces[i]))
        #Cap velocities at max_vel
        velocities = map(lambda velocity: velocity if np.linalg.norm(velocity) < bird_max_vel else np.divide(velocity, np.linalg.norm(velocity)/bird_max_vel), velocities)
        #Update our flocks with the new positions
        agents[i] = map(lambda position, velocity: (position, velocity), positions, velocities)
        newpolys = map(lambda bird: gen_polygon(bird[0], bird[1]), flock)
        for j,poly in enumerate(newpolys):
            patches[i][j].set_xy(poly)
            npatches.append(patches[i][j])

    return npatches

""" This function will randomly initialize the birds. 
It produces num_flocks different flocks each with num_bird_in_flock birds"""
def random_init_birds(num_flocks, num_birds_in_flock):
    if num_flocks > len(flock_colors):
        raise "Too many flocks!, Add more colors!"
    flocks = []
    for i in xrange(num_flocks):
        flocks.append([])
        for j in xrange(num_birds_in_flock):
            #Each bird is represented by a pair of vectors [position, velocity]
            #We randomly generate these positions and vectors
            flocks[i].append([(95*random.random()+5, 95*random.random()+5), generate_random_velocity_vector(bird_max_vel)])
    return flocks

#Generates a random velocity vector with magnitude at most max_vel
def generate_random_velocity_vector(max_vel):
    x,y = max_vel, max_vel
    while np.linalg.norm([x,y]) > max_vel:
        x = (random.random()-0.5)*2*max_vel
        y = (random.random()-0.5)*2*max_vel
    return (x,y)


"""Generates the icon for a single bird on the plot. Don't worry about this stuff ;)"""
def gen_polygon(x, v):
    #Scales the size of a bird icon
    c = 2.0
    if x[0] > 100 or x[0] < 0 or x[1] > 100 or x[1] < 0:
        raise ValueError("Bird out of bounds at " + str(x[0]) + "," + str(x[1]))
    vn = np.linalg.norm(v)
    return [[x[0] + (v[0]/vn)*(2*c/3), x[1] + (v[1]/vn)*(2*c/3)],
            [x[0] - (v[0]/vn)*(c/3) + (v[1]/vn)*(c/4),   x[1] - (v[1]/vn)*(2*c/3) - (v[0]/vn)*(c/4)],
            [x[0] - (v[0]/vn)*(c/3) - (v[1]/vn)*(c/4),   x[1] - (v[1]/vn)*(2*c/3) + (v[0]/vn)*(c/4)]]



Alright, now that we've set up the infrastructure, let's go ahead and run our first simulation!

In [5]:
#Setup figure
#Don't change this!
fig = plt.figure()
ax = plt.axes(xlim=(0, 100), ylim=(0, 100))
plt.tight_layout()

#Colors
#Feel free to change!
flock_colors = ['b','g','c','grey', 'darkviolet','gold']

#Set number of blocks and number of birds in each flock
#Don't set these too high or else perfomance will suffer!
#These need to ints
num_flocks = 1
num_birds_in_flock = 20

#Flocking settings -try changing these and see what happens! 
#ALL OF THESE SHOULD BE FLOATS!
bird_max_vel = 25.0
sight_radius = 10.0

#Rendering speeds
interval = 20.0

#Populates and stores birds in the simulation
#Each flock is a row of birds, each bird is a pair of vectors [position, velocity]
flocks = random_init_birds(num_flocks, num_birds_in_flock)
patches = [[plt.Polygon(gen_polygon(flocks[i][j][0], flocks[i][j][1]), alpha=1, color=flock_colors[i]) for j in xrange(len(flocks[i]))] for i in xrange(len(flocks))]

def init():
    return ax

#Update the frame
def animate(i):
    npatches = []
    global flocks
    global patches

    #If this is the first frame, initialize the view!
    if i==0:
        #Add all of our birds to the window
        for flock in patches:
            for patch in flock:
                plt.gca().add_patch(patch)
                npatches.append(patch)
        return npatches

    #Calculate the wall avoidance force
    wall_force = avoid_wall_force(flocks)
    #Calculate the motive force that tries to keep the birds moving
    mot_force = motive_force(flocks)
    #Sum the forces to obtain the total force on each bird!
    total_force = sum_arrays(wall_force,mot_force)

    #Now we update the position and velocity of the birds according to the force
    updatePositionVelocity(flocks, total_force, patches, npatches)

    return npatches

#Animate!
anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=10000, 
                               interval=interval,
                               blit=True)

plt.show()

Well that wasn't very interesting. The birds just ignored each other and went on more or less straight paths, but at least we managed to keep them in the box! Now we can add some more behaviors. Let's think about the kinds of behaviors that define a flock. What simple forces and rules can we add to produce an emergent system that acts like a flock?

It turns out there is a subfield of scientists that research just these kinds of concerns, and flocking is extremely well studied.

There are only three individual "fundamental forces" that combine produce flocking behavior!

For realism, let's introduce the idea of a 'sight range' and 'separation distance.' Birds can only directly react to things within their sight range, and will try to stay away from other birds within the separation distance. The separation distance also helps the visualization, since it's easier to tell what's going on when all the birds aren't overlapping.

<img width="225px" src="images/detection_circles.gif">

The three fundamental flocking forces are [drumroll please] . . . 

### Cohesion

In order for a flock to be, well, a flock, the birds need to stay close together! The cohesion force promotes staying together by adding a force that pushes a bird toward the average location of the birds within its *sight range* (i.e. the center of the visible flock). To produce this force, we simply average the positions of the birds that a single bird can see and make a force vector that points toward that average.

<img width="225px" src="images/cohesion.gif">

### Alignment

Flocks move in a coordinated manner, as is clearly visible from the gifs in the introduction. This coordination is produced by every member of the flock trying to stay in alignment with its neighbors. We can simulate this by adding a force that pushes the velocity of a bird towards the average velocity of all the birds around it.

<img width="225px" src="images/alignment.gif">

### Separation

Wait, didn't we just say flocks needed to say together? Well, yes, but of course birds (and flocking animals in general) don't want to be right on top of each other. We all need personal space! We can simulate this by adding a force that pushes a bird away from its neighbors that are within the separation distance.

<img width="225px" src="images/separation.gif">



We now have a set of forces that can produce flocking. But wait, you might say! While we know the general form of these forces, we don't have exact descriptions! Should the force scale with distance, or the square of distance, or not at all? And how strong should the alignment force be compared to the cohesion and separation forces?

The answer is we don't precisely know! We tune the parameters to make the model behave realistically. One possible setting of these parameters is used below, but we encourage you to play around with the parameters in the section below and see the results!

**The next box defines functions to calculate the three forces we described above. Run it before proceeding.**

In [12]:
"""
Generates forces that tend to bring birds closer together, at least until they get too close for comfort
"""
def cohesion_force(flocks):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    c = 1.0

    for i in xrange(len(flocks)):
        counts = [1 for k in xrange(len(flocks[i]))]
        for j in xrange(len(flocks[i])):
            for k in xrange(j+1, len(flocks[i])):
                #Calculate offset vector between the two birds
                f = np.subtract(flocks[i][j][0], flocks[i][k][0])
                dist = np.linalg.norm(f)
                #If the birds are close enough to see each other but not too close, push them towards each other
                if dist > individual_separation and dist < sight_radius:
                    forces[i][j] = np.add(forces[i][j], np.multiply(-cohesion*c, f))
                    forces[i][k] = np.add(forces[i][k], np.multiply(cohesion*c, f))
                    counts[i] += 1
                    counts[k] += 1
        forces[i] = np.divide(forces[i],np.transpose([counts, counts]))

    return forces

"""
Generates a force that aligns the movement of nearby members of the flock
"""
def alignment_force(flocks):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    c = 10.0

    for i in xrange(len(flocks)):
        for j in xrange(len(flocks[i])):
            for k in xrange(j+1, len(flocks[i])):
                dist = np.linalg.norm(np.subtract(flocks[i][j][0], flocks[i][k][0]))
                #If the birds are close enough to see each other, push them towards alignment
                if dist < sight_radius:
                    #Force vector is proportional to the difference in velocity vectors
                    f = np.subtract(flocks[i][k][1], flocks[i][j][1])
                    forces[i][j] = np.add(forces[i][j], np.multiply(alignment*c/dist, f))
                    forces[i][k] = np.add(forces[i][k], np.multiply(-alignment*c/dist, f))

    return forces

""" Birds don't want to be too close together! The separation force drives them apart if they are closer
than individual_separation apart"""
def separation_force(flocks):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    c =500.0

    for i in xrange(len(flocks)):
        for j in xrange(len(flocks[i])):
            for k in xrange(j+1, len(flocks[i])):
                f = np.subtract(flocks[i][k][0], flocks[i][j][0])
                dist = np.linalg.norm(f)
                #If the birds are too close, push them apart
                if dist < individual_separation:
                    forces[i][j] = np.add(forces[i][j], np.multiply(-c/dist, f))
                    forces[i][k] = np.add(forces[i][k], np.multiply(c/dist, f))

    return forces

Now let's do that simulation again, but with the extra forces we just added.

Note that only birds of the same color will flock with each other! (These birds are kind of discriminatory).

Play with the parameters defined in the top section of the code and see how they affect the flocks that form. In particular, check out how the flock size and individual_separation interact as the flocks get larger.

In [11]:
#Setup figure
#Don't change this!
fig = plt.figure()
ax = plt.axes(xlim=(0, 100), ylim=(0, 100))
plt.tight_layout()

#Colors
#Feel free to change!
flock_colors = ['b','g','c','grey', 'darkviolet','gold']

#Set number of blocks and number of birds in each flock
#Don't set these too high or else perfomance will suffer!
#These need to ints
num_flocks = 2
num_birds_in_flock = 15

#Flocking settings -try changing these and see what happens! 
#ALL OF THESE SHOULD BE FLOATS!
bird_max_vel = 25.0
sight_radius = 15.0
individual_separation = 1.0
cohesion = 1.0
alignment = 1.0

#Rendering speeds
interval = 20.0

#Populates and stores birds in the simulation
#Each flock is a row of birds, each bird is a pair of vectors [position, velocity]
flocks = random_init_birds(num_flocks, num_birds_in_flock)
patches = [[plt.Polygon(gen_polygon(flocks[i][j][0], flocks[i][j][1]), alpha=1, color=flock_colors[i]) for j in xrange(len(flocks[i]))] for i in xrange(len(flocks))]

def init():
    return ax

#Update the frame
def animate(i):
    npatches = []
    global flocks
    global patches

    #If this is the first frame, initialize the view!
    if i==0:
        #Add all of our birds to the window
        for flock in patches:
            for patch in flock:
                plt.gca().add_patch(patch)
                npatches.append(patch)
        return npatches

    #Calculate the wall avoidance force
    wall_force = avoid_wall_force(flocks)
    #Calculate the motive force that tries to keep the birds moving
    mot_force = motive_force(flocks)
    #Calculate the cohesion force
    coh_force = cohesion_force(flocks)
    #Calculate the alignment force
    align_force = alignment_force(flocks)
    #Calculate the separation force
    sep_force = separation_force(flocks)
    #Sum the forces to obtain the total force on each bird!
    total_force = sum_arrays(wall_force,mot_force, coh_force, align_force, sep_force)

    #Now we update the position and velocity of the birds according to the force
    updatePositionVelocity(flocks, total_force, patches, npatches)

    return npatches

#Animate!
anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=10000, 
                               interval=interval,
                               blit=True)

plt.show()

ValueError: Bird out of bounds at 58.1544811856,100.024081197

Finally, just for fun let's add a hawk into the simulation

The birds all try to run away from the hawk, while the hawk chases the closest bird that it can see.
We simulate this by adding a force that pushes the birds away from the hawk if the hawk is within the bird's sight radius, while adding a force to the hawk that aims it toward the closest bird it can see.

If you have time, try changing the hawk AI so it is smarter when pursuing a bird (maybe it tries to predict the bird's position according to its velocity instead of just going towards its current position) or implementing a way for the hawk to "eat" [remove from the field] birds that it gets close enough to.


In [1]:
#Setup figure
#Don't change this!
fig = plt.figure()
ax = plt.axes(xlim=(0, 100), ylim=(0, 100))
plt.tight_layout()

#Colors
#Feel free to change!
flock_colors = ['b','g','c','grey', 'darkviolet','gold']
pred_color = 'r'

#Set number of blocks and number of birds in each flock
#Don't set these too high or else perfomance will suffer!
#These need to ints
num_flocks = 4
num_birds_in_flock = 15

#Predator settings
#Note: With the way the simulation is set up, the chases tend to most interesting when the predator is a little slower
#than the birds but can see somewhat further
pred_sight_radius = 20.0
pred_max_vel = 20.0
#Set pred_chase to zero to have the predator ignore the birds
pred_chase = 1

#Flocking settings -try changing these and see what happens! 
#ALL OF THESE SHOULD BE FLOATS!
bird_max_vel = 25.0
sight_radius = 10.0
individual_separation = 1.0
cohesion = 1.0
alignment = 1.0
predator_avoidance = 1.0

#Rendering speeds
interval = 20.0

#Populates and stores birds in the simulation
#Each flock is a row of birds, each bird is a pair of vectors [position, velocity]
flocks = random_init_birds(num_flocks, num_birds_in_flock)
patches = [[plt.Polygon(gen_polygon(flocks[i][j][0], flocks[i][j][1]), alpha=1, color=flock_colors[i]) for j in xrange(len(flocks[i]))] for i in xrange(len(flocks))]
#Oh no it's a hawk!
predator = [[95*random.random()+5, 95*random.random()+5], generate_random_velocity_vector(bird_max_vel)]
pred_patch = plt.Polygon(gen_polygon(predator[0], predator[1]), alpha=1, color=pred_color)


"""Produces the effect of a predator on the motion of the birds and on the motion of the predator itself
The predator will track towards the closest bird in its sight radius
The birds always try to flee the predator"""
def predator_force(flocks, predator):
    forces = np.zeros((len(flocks), len(flocks[0]), 2))
    pred_force = [0,0]
    c = 500.0

    min_distance_bird = [100, None]
    for i in xrange(len(flocks)):
        for j in xrange(len(flocks[i])):
            f = np.subtract(predator[0], flocks[i][j][0])
            dist = np.linalg.norm(f)
            if dist < min_distance_bird[0] and dist < pred_sight_radius:
                min_distance_bird = [dist, -f]
            if dist < sight_radius:
                forces[i][j] = np.add(forces[i][j], np.multiply(-predator_avoidance*c/dist, f))

    if min_distance_bird[1] != None and pred_chase > 0:
        pred_force = np.multiply(10.0,min_distance_bird[1])

    return pred_force,forces

#Helper function to update the predator's position
def updatePositionVelocityForPredator(predator, force, pred_patch, npatches):
    pred_patch.set_xy(gen_polygon(predator[0], predator[1]))
    predator[0] = np.add(predator[0], np.multiply(interval/1000.0, predator[1]))
    predator[1] = np.add(predator[1], np.multiply(interval/1000.0, force))
    #Cap predator velocity also!
    predator[1] = predator[1] if np.linalg.norm(predator[1]) < pred_max_vel else np.divide(predator[1], np.linalg.norm(predator[1])/pred_max_vel)
    npatches.append(pred_patch)

    return npatches


def init():
    return ax,

#Update the frame
def animate(i):
    npatches = []
    global flocks
    global patches
    global predator
    global pred_patch
    
    #If this is the first frame, initialize the view!
    if i==0:
        #Add all of our birds to the window
        for flock in patches:
            for patch in flock:
                plt.gca().add_patch(patch)
                npatches.append(patch)
        #Add the predator to the window
        plt.gca().add_patch(pred_patch)
        npatches.append(pred_patch)
        return npatches

    #Calculate the wall avoidance force
    wall_force = avoid_wall_force(flocks)
    #Calculate the motive force that tries to keep the birds moving
    mot_force = motive_force(flocks)
    #Calculate the cohesion force
    coh_force = cohesion_force(flocks)
    #Calculate the alignment force
    align_force = alignment_force(flocks)
    #Calculate the separation force
    sep_force = separation_force(flocks)
    #Calculate the effect of the predator
    pred_force, flee_force = predator_force(flocks, predator)
    #Sum the forces to obtain the total force on each bird!
    total_force = sum_arrays(wall_force,mot_force, coh_force, align_force, sep_force, flee_force)

    #Now we update the position and velocity of the birds according to the force
    updatePositionVelocity(flocks, total_force, patches, npatches)
    #Do the same for the predator
    pred_wall_force = avoid_wall_force([[predator]])[0][0]
    updatePositionVelocityForPredator(predator, np.add(pred_wall_force,pred_force), pred_patch, npatches)


    return npatches

#Animate!
anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=10000, 
                               interval=interval,
                               blit=True)

plt.show()

NameError: name 'plt' is not defined